In [1]:
%pip install pypdf


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lark


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
import re
import os

data_path = '../../data/UNVXIS.05 Enable Internal Sales Orders.pdf'

loader = PyPDFLoader(data_path)

pages = []
async for page in loader.alazy_load():
    pages.append(page)

def preprocess_content(content):
    # Remove the specific phrase from the content
    return re.sub(r'Enable Internal Sales Orders \(Internal Sales Orders\) \d+ \(\d+\)', '', content)

def split_by_headers(documents, header_pattern, specific_headers):
    split_docs = []
    last_seen_header = None
    for doc in documents:
        # Preprocess the content to remove the specific phrase
        content = preprocess_content(doc.page_content)
        chunks = re.split(f'({header_pattern})', content)
        chunks = [chunk.strip() for chunk in chunks if chunk.strip()]
        for chunk in chunks:
            if chunk in specific_headers:
                last_seen_header = chunk
            else:
                split_docs.append(
                    Document(
                        page_content=chunk,
                        metadata={'header': last_seen_header}
                    )
                )
    return split_docs

# Define the specific headers you want to capture
specific_headers = ["Overview", "Tasks", "Setup", "Additional Information"]

# Define the header pattern (adjust the pattern based on your headers)
header_pattern = r'\b(?:' + '|'.join(specific_headers) + r')\b'

# Split the document by headers
docs = split_by_headers(pages, header_pattern, specific_headers)

# Print the first chunk's content and metadata
docs = docs[2:]

for doc in docs:
    #print(doc.page_content)
    print(doc.metadata)
    print('-------------------')
print(len(docs))

{'header': 'Overview'}
-------------------
{'header': 'Tasks'}
-------------------
{'header': 'Tasks'}
-------------------
{'header': 'Tasks'}
-------------------
{'header': 'Tasks'}
-------------------
{'header': 'Tasks'}
-------------------
{'header': 'Tasks'}
-------------------
{'header': 'Tasks'}
-------------------
{'header': 'Setup'}
-------------------
{'header': 'Additional Information'}
-------------------
{'header': 'Additional Information'}
-------------------
11


In [4]:
from dotenv import load_dotenv
import os
load_dotenv()

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
ASTRA_DB_API_KEY = os.getenv('ASTRA_DB_API_KEY')
ASTRA_DB_ENDPOINT = os.getenv('ASTRA_DB_ENDPOINT')
ASTRA_DB_KEYSPACE = os.getenv('ASTRA_DB_KEYSPACE')

model = ChatOpenAI(model='gpt-4o')

embeddings = OpenAIEmbeddings()

from langchain_community.vectorstores import AstraDB

vectorstore = AstraDB(
    embedding=embeddings,
    token=ASTRA_DB_API_KEY,
    api_endpoint=ASTRA_DB_ENDPOINT,
    collection_name="not_metadata",

)

/tmp/ipykernel_364733/1536256942.py:19: LangChainDeprecationWarning: The class `AstraDB` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-astradb package and should be used instead. To use it run `pip install -U :class:`~langchain-astradb` and import as `from :class:`~langchain_astradb import AstraDBVectorStore``.
  vectorstore = AstraDB(


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
from langchain_community.vectorstores import AstraDB

vectorstore = AstraDB(
    embedding=embeddings,
    token=ASTRA_DB_API_KEY,
    api_endpoint=ASTRA_DB_ENDPOINT,
    collection_name="not_metadata",

)

In [ ]:
vectorstore

In [ ]:
import uuid

#doc_ids = [str(uuid.uuid4()) for _ in docs]

In [ ]:
#vectorstore.add_documents(documents=docs, doc_ids=doc_ids)

In [ ]:
import os
from astrapy import DataAPIClient

# Initialize the client and get a "Database" object
client = DataAPIClient(ASTRA_DB_API_KEY)
database = client.get_database(ASTRA_DB_ENDPOINT)
collection = database.get_collection("not_metadata")

# Retrieve all documents from the collection
documents = collection.find({})

# Print the retrieved documents
for doc in documents:
    print(doc["metadata"])
    print('-------------------')

{'header': 'Additional Information'}
-------------------
{'header': 'Tasks'}
-------------------
{'header': 'Tasks'}
-------------------
{'header': 'Overview'}
-------------------
{'header': 'Tasks'}
-------------------
{'header': 'Tasks'}
-------------------
{'header': 'Tasks'}
-------------------
{'header': 'Tasks'}
-------------------
{'header': 'Tasks'}
-------------------
{'header': 'Additional Information'}
-------------------
{'header': 'Setup'}
-------------------


In [ ]:
documents = collection.find({})

for doc in documents:
    print(doc["metadata"])
    print(doc["content"])
    print('-------------------')

{'header': 'Additional Information'}
Historic Actual Cost (HAC) and Goods-in-transit (GIT) 
The Historic Actual Cost (HAC) calculation is not currently supported in combination with the 
Goods in Transit (GIT) functionality. 
 
Line Charges 
Header charges have been setup to enable the addition of header charges at customer order 
that will be copied to the linked purchase order when the customer order is confirmed. 
It is also possible to add line charges, however this has not been configured in this IPC 
solution. 
 
Supply Chain Orders 
If supply chain orders are used, the specific settings for supply chain orders must also be set. 
Note: If applicable, see Preconfigured Data – Supply Chain Orders. 
 
Transportation lead time 
In 'Supplier. Connect Transp Lead Times' (PPS010), both fields 'Transportation lead time 1' 
(TLE1) and 'Transportation lead time 2' (TLE2) are enabled if the supplier is type 7 (internal). 
This allows for the suppliers internal lead time to be taken into acc

In [ ]:
documents = collection.find({})

len_tasks = 0
for doc in documents:
    if doc["metadata"]["header"] == "Tasks":
        #print(doc["metadata"])
        print(doc["content"])
        print('\n')
        print('-------------------')
        len_tasks +=1

print(len_tasks)

Manage PO Settings 
1. Define PO Type 
‘Purchase Order Type. Open’ (PPS095) 
 Order type: I01 (purchase order category 20) 
o 040 Goods receiving method: (direct put-away) 
o 360 PO status – auto confirmation: 0, manual confirmation 
o 710 Automatic printout of PO documents: No (Deactivated) 
Note: For IPC data, see Preconfigured Data – Internal Sales Orders. 
 
2. Define Suppliers 
‘Supplier Group. Open’ (CRS150) 
 Supplier group: INT 
 
‘Supplier. Open’ (CRS620) 
 Supplier: YINT001 
o Supplier type: 7-Internal 
 
‘Supplier. Define Purchase & Financial’ (CRS624) 
 Supplier: YINT001 
o Order type: I01 (E) 
o Delivery terms: FOB (E) 
o Shipment advice: 1-Rpe. via EDI (F) 
o Facility (Internal Sales): A01 (G) 
o Supplying whs (Internal Sales): 001 (G) 
 
‘Supplier. Connect our Customer Number’ (CRS680) (option 17 in CRS620) 
 Supplier: YINT001  
o Warehouse: 910 
o Customer number: YINT910 
 
‘Purchase Agreement. Open’ (PPS100) 
 Agreement: INT0001 
 
3. Define PO Charges 
‘Setting

In [ ]:
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

metadata_field_info = [
    AttributeInfo(
        name='header',
        description="The 'header' of the page. The 'header' is one of ['Overview', 'Tasks', 'Setup', 'Additional information']",
        type="string",
    ),
]

document_content_description = "The pages"

retriever = SelfQueryRetriever.from_llm(
    model, 
    vectorstore, 
    document_content_description, 
    metadata_field_info, 
    verbose=True,
    search_kwargs={"k": 10},
)

results = retriever.invoke(
    "Retrieve all documents where the 'header' is 'Tasks'. You should give me 7 pages in return"
)

In [ ]:
len(results)

7

In [ ]:
for doc in results:
    print(doc.metadata)
    print('-------------------')

{'header': 'Tasks'}
-------------------
{'header': 'Tasks'}
-------------------
{'header': 'Tasks'}
-------------------
{'header': 'Tasks'}
-------------------
{'header': 'Tasks'}
-------------------
{'header': 'Tasks'}
-------------------
{'header': 'Tasks'}
-------------------
